In [41]:
import pandas as pd
import numpy as np

# 데이터 불러오기
data1 = pd.read_csv('WallThining_data.csv')
data2 = pd.read_csv('WallThinning_3050_data.csv')
data = pd.concat([data1, data2], axis=0)
data = data.drop('Moment', axis=1).reset_index(drop=True)
# 데이터 확인
print(data.head())

  Thining Part Moment Direction Thining Rate Boundary Type  Inner Pressure  \
0        Extra              IPO          20%    Limit Load          22.070   
1        Extra              IPO          20%    Limit Load          19.863   
2        Extra              IPO          20%    Limit Load          17.656   
3        Extra              IPO          20%    Limit Load          15.449   
4        Extra              IPO          20%    Limit Load          13.242   

   Multiplier  
0    0.754368  
1    0.889164  
2    1.000607  
3    1.022539  
4    0.970476  


In [42]:
len(data1), len(data2), len(data)

(594, 396, 990)

In [43]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max Scaler 객체 생성
scaler = MinMaxScaler()

# 'Inner Pressure' 열 선택 및 Reshape (-1, 1) 필요 (scaler는 2D 배열을 요구함)
inner_Pressure = data['Inner Pressure'].values.reshape(-1, 1)

# Min-Max Normalization 적용
data['Inner Pressure'] = scaler.fit_transform(inner_Pressure)

# 결과 확인
print(data.head())


  Thining Part Moment Direction Thining Rate Boundary Type  Inner Pressure  \
0        Extra              IPO          20%    Limit Load             1.0   
1        Extra              IPO          20%    Limit Load             0.9   
2        Extra              IPO          20%    Limit Load             0.8   
3        Extra              IPO          20%    Limit Load             0.7   
4        Extra              IPO          20%    Limit Load             0.6   

   Multiplier  
0    0.754368  
1    0.889164  
2    1.000607  
3    1.022539  
4    0.970476  


In [44]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Categorical 변수 인코딩
# le = LabelEncoder()
# data['Thining Part'] = le.fit_transform(data['Thining Part'])
# data['Moment Direction'] = le.fit_transform(data['Moment Direction'])
# data['Boundary Type'] = le.fit_transform(data['Boundary Type'])

data = pd.get_dummies(data, columns=['Thining Part','Moment Direction','Boundary Type'])

# Numerical 변수 전처리 (Inner pressure)
data['Thining Rate'] = data['Thining Rate'].str.rstrip('%').astype('int')

# 데이터 분할
X = data.drop('Multiplier', axis=1)
y = data['Multiplier']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [47]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

models = {
    'MLP': Sequential([
        Dense(64, activation='relu', input_shape=(10,)),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ]),
    'CNN': Sequential([
        # Conv1D는 3D 입력이 필요하므로 (샘플 수, 타임스텝, 특성) 형태로 변환 필요
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(10, 1)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(1, activation='linear')
    ]),
    'RNN': Sequential([
        SimpleRNN(50, activation='relu', input_shape=(10, 1)),
        Dense(1, activation='linear')
    ]),
    'LSTM': Sequential([
        LSTM(50, activation='relu', input_shape=(10, 1)),
        Dense(1, activation='linear')
    ]),
    'GRU': Sequential([
        GRU(50, activation='relu', input_shape=(10, 1)),
        Dense(1, activation='linear')
    ])
}

# MAPE와 SMAPE 함수 정의
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true) + np.abs(y_pred)) / 2)) * 100


# 모델 컴파일 및 학습
for name, model in models.items():
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    if name == 'MLP':
        model.fit(X_train, y_train, validation_split=0.2, epochs=10, callbacks=[early_stop], verbose=0)
        y_pred = model.predict(X_test)
    else:
        X_train_transformed = X_train.to_numpy().reshape((X_train.shape[0], X_train.shape[1], 1))
        model.fit(X_train_transformed, y_train, validation_split=0.2, epochs=10, callbacks=[early_stop], verbose=0)
        X_test_transformed = X_test.to_numpy().reshape((X_test.shape[0], X_test.shape[1], 1))
        y_pred = model.predict(X_test_transformed)

    # 실제 값과 예측 값의 차이를 계산하여 평가 지표를 구함
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)   
    r2 = r2_score(y_test, y_pred)

    print(f'{name} - MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}, SMAPE: {smape:.4f}, R2: {r2:.4f}')

MLP - MAE: 0.0842, MSE: 0.0109, RMSE: 0.1044, MAPE: 28.7091, SMAPE: 25.4094, R2: 0.5681
CNN - MAE: 0.0977, MSE: 0.0133, RMSE: 0.1151, MAPE: 25.5396, SMAPE: 25.4209, R2: 0.4748
RNN - MAE: 0.1132, MSE: 0.0184, RMSE: 0.1355, MAPE: 25.6938, SMAPE: 22.4491, R2: 0.2730
LSTM - MAE: 0.1107, MSE: 0.0189, RMSE: 0.1374, MAPE: 25.5531, SMAPE: 21.9268, R2: 0.2517
GRU - MAE: 0.1161, MSE: 0.0207, RMSE: 0.1438, MAPE: 24.7511, SMAPE: 21.5837, R2: 0.1812


In [55]:
data_60 = data[data['Thining Rate'] == 60]
X_test = data_60.drop('Multiplier', axis=1)
y_test = data_60['Multiplier']

# data_60 = data[data['Thining Rate'] == 60]


# 모델 컴파일 및 학습
for name, model in models.items():
    if name == 'MLP':
        y_pred = model.predict(X_test)
    else:
        X_test_transformed = X_test.to_numpy().reshape((X_test.shape[0], X_test.shape[1], 1))
        y_pred = model.predict(X_test_transformed)

    # 실제 값과 예측 값의 차이를 계산하여 평가 지표를 구함
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)   
    r2 = r2_score(y_test, y_pred)
    
    print(f'{name} - MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}, SMAPE: {smape:.4f}, R2: {r2:.4f}')

MLP - MAE: 0.0940, MSE: 0.0150, RMSE: 0.1224, MAPE: 29.4476, SMAPE: 24.9550, R2: 0.2183
CNN - MAE: 0.0960, MSE: 0.0132, RMSE: 0.1147, MAPE: 25.0184, SMAPE: 24.8561, R2: 0.3126
RNN - MAE: 0.1067, MSE: 0.0174, RMSE: 0.1320, MAPE: 25.5343, SMAPE: 21.3825, R2: 0.0901
LSTM - MAE: 0.1087, MSE: 0.0173, RMSE: 0.1316, MAPE: 23.9215, SMAPE: 20.1908, R2: 0.0958
GRU - MAE: 0.1071, MSE: 0.0179, RMSE: 0.1338, MAPE: 22.9761, SMAPE: 19.7122, R2: 0.0658


In [5]:
import pandas as pd
import numpy as np

# 데이터 불러오기
data = pd.read_csv('WallThinning_3050_data.csv')
data = data.drop('Moment', axis=1)
# 데이터 확인
print(data.head())

  Thining Part Moment Direction Thining Rate      Boundary Type  \
0        Intra              IPO          30%  Elastic Shakedown   
1        Intra              IPO          30%  Elastic Shakedown   
2        Intra              IPO          30%  Elastic Shakedown   
3        Intra              IPO          30%  Elastic Shakedown   
4        Intra              IPO          30%  Elastic Shakedown   

   Inner Pressure  Multiplier  
0           0.000    0.356588  
1           2.207    0.396221  
2           4.414    0.445776  
3           6.621    0.509500  
4           8.828    0.594469  


In [6]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max Scaler 객체 생성
scaler = MinMaxScaler()

# 'Inner Pressure' 열 선택 및 Reshape (-1, 1) 필요 (scaler는 2D 배열을 요구함)
inner_Pressure = data['Inner Pressure'].values.reshape(-1, 1)

# Min-Max Normalization 적용
data['Inner Pressure'] = scaler.fit_transform(inner_Pressure)

# 결과 확인
print(data.head())


  Thining Part Moment Direction Thining Rate      Boundary Type  \
0        Intra              IPO          30%  Elastic Shakedown   
1        Intra              IPO          30%  Elastic Shakedown   
2        Intra              IPO          30%  Elastic Shakedown   
3        Intra              IPO          30%  Elastic Shakedown   
4        Intra              IPO          30%  Elastic Shakedown   

   Inner Pressure  Multiplier  
0             0.0    0.356588  
1             0.1    0.396221  
2             0.2    0.445776  
3             0.3    0.509500  
4             0.4    0.594469  


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

data = pd.get_dummies(data, columns=['Thining Part','Moment Direction','Boundary Type'])

# Numerical 변수 전처리 (Inner pressure)
data['Thining Rate'] = data['Thining Rate'].str.rstrip('%').astype('int')

# 데이터 분할
X = data.drop('Multiplier', axis=1)
y = data['Multiplier']

In [9]:
# 모델 컴파일 및 학습
for name, model in models.items():
    if name == 'MLP':
        y_pred = model.predict(X)
    else:
        X_transformed = X.to_numpy().reshape((X.shape[0], X.shape[1], 1))
        y_pred = model.predict(X_transformed)

    # 실제 값과 예측 값의 차이를 계산하여 평가 지표를 구함
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y, y_pred)
    smape = symmetric_mean_absolute_percentage_error(y, y_pred)   
    r2 = r2_score(y, y_pred)

    print(f'{name} - MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}, SMAPE: {smape:.4f}, R2: {r2:.4f}')

MLP - MAE: 0.2400, MSE: 0.0866, RMSE: 0.2944, MAPE: 35.9376, SMAPE: 45.3584, R2: -2.5883
CNN - MAE: 0.0981, MSE: 0.0150, RMSE: 0.1225, MAPE: 25.5816, SMAPE: 22.3467, R2: 0.3789
RNN - MAE: 0.1229, MSE: 0.0216, RMSE: 0.1470, MAPE: 24.5112, SMAPE: 23.7320, R2: 0.1047
LSTM - MAE: 0.1168, MSE: 0.0213, RMSE: 0.1459, MAPE: 24.5965, SMAPE: 21.0523, R2: 0.1186
GRU - MAE: 0.1201, MSE: 0.0208, RMSE: 0.1444, MAPE: 22.9089, SMAPE: 20.8052, R2: 0.1370
